## Translating and detecting the language of desc column 

In [47]:
# %pip install googletrans==4.0.0rc1

In [48]:
#%pip install langdetect

In [49]:
#%pip install textblob

In [50]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from pandarallel import pandarallel
from textblob import TextBlob
from googletrans import LANGUAGES, Translator
from langdetect import detect


pandarallel.initialize(progress_bar=True)
tqdm.pandas()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [51]:
pd.__version__

'2.0.0rc0'

In [52]:
df = pd.read_json('as24_cars.json')
pd.options.display.max_columns = 100

In [53]:
df.sample(3)

,make_model,short_description,make,model,location,price,Body type,Type,Doors,Country version,Offer number,Warranty,Mileage,First registration,Gearbox,Fuel type,Colour,Paint,desc,seller,Seats,Power,Engine size,Gears,CO₂-emissions,Manufacturer colour,Drivetrain,Cylinders,Fuel consumption,\nComfort & Convenience\n,\nEntertainment & Media\n,\nSafety & Security\n,\nExtras\n,Empty weight,Model code,General inspection,Last service,Full service history,Non-smoker vehicle,Emission class,Emissions sticker,Upholstery colour,Upholstery,Production date,Previous owner,Other fuel types,Power consumption,Energy efficiency class,CO₂-efficiency,Fuel consumption (WLTP),CO₂-emissions (WLTP),Available from,Taxi or rental car,Availability,Last timing belt change,Electric Range (WLTP),Power consumption (WLTP),Battery Ownership
2283,Mercedes-Benz A 35 AMG,4M*Fahrass*Pano*Kamera*Burmester*Keyles,\nMercedes-Benz\n,"[\n, A 35 AMG ,\n]","Merowinger Str. 7-9, 70736 Fellbach, DE","€ 48,900.-","[\n, Sedan, \n]","[\n, Used, \n]","[\n, 4, \n]","[\n, Germany, \n]","[\n, 30220588, \n]","[\n, Yes, \n]","11,250 km",10/2020,[\nAutomatic\n],Super E10 95,Grey,Metallic,"[Fahrzeug-Nr. für Kundenanfragen: 30220588, *e...",Dealer,"[\n, 5, \n]",[\n225 kW (306 hp)\n],"[\n1,991 cc\n]",NaN,188 g/km (comb.),Mountaingrau,"[\n, 4WD, \n]",NaN,"[[7.4 l/100 km (comb.)], [8.9 l/100 km (city)]...","[Air conditioning, Armrest, Automatic climate ...","[Android Auto, Apple CarPlay, Bluetooth, Digit...","[ABS, Adaptive Cruise Control, Alarm system, B...","[Alloy wheels, Ambient lighting, Automatically...",NaN,"[\n, 2222/AJD, \n]",New,NaN,Yes,Yes,Euro 6d-TEMP,4 (Green),Black,Full leather,NaN,"[[11,250 km, 10/2020], 3]",NaN,NaN,E,Calculated on basis of measured CO₂-emissions ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1481,Mercedes-Benz G 350,DESIGNO*AMG SPORT*DISTR*PANO*STNDH*AHK*HK7,\nMercedes-Benz\n,"[\n, G 350 ,\n]","Schulstraße 117, 73066 Uhingen, DE","€ 93,490.-","[\n, Off-Road/Pick-up, \n]","[\n, Used, \n]","[\n, 5, \n]","[\n, Germany, \n]","[\n, 1X27ASU5783, \n]",NaN,"36,700 km",04/2017,[\nAutomatic\n],Diesel (Particle filter),Black,Metallic,[IN DIESEM JAHR SIND WIR VON AUTOBILD UNTER DE...,Dealer,"[\n, 5, \n]",[\n180 kW (245 hp)\n],"[\n2,987 cc\n]",[\n7\n],262 g/km (comb.),Sonderl. Obsidianschwarz Metal,"[\n, 4WD, \n]",[\n6\n],"[[9.9 l/100 km (comb.)], [11.1 l/100 km (city)...","[Air conditioning, Armrest, Automatic climate ...","[Android Auto, Apple CarPlay, Bluetooth, CD pl...","[ABS, Adaptive Cruise Control, Adaptive headli...","[Alloy wheels (19""), Automatically dimming int...","[\n2,610 kg\n]","[\n, 1313/ERT, \n]",New,NaN,Yes,Yes,Euro 6,4 (Green),NaN,Full leather,NaN,"[[36,700 km, 04/2017], 1]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25068,Hyundai i30,1.0 TGDI Klass,\nHyundai\n,"[\n, i30 ,\n]","CARRETERA BILBAO GALDAKAO KM. 6, 48960 BILBAO...","€ 15,290.-","[\n, Station wagon, \n]","[\n, Used, \n]","[\n, 5, \n]","[\n, Spain, \n]","[\n, 7286952, \n]","[\n, 12 months, \n]","5,790 km",08/2020,[\nManual\n],Gasoline,Red,NaN,"[ , Precio al contado: 18490 euros, , Para co...",Dealer,NaN,[\n88 kW (120 hp)\n],"[\n1,000 cc\n]",NaN,NaN,Rojo,NaN,NaN,"[[6.1 l/100 km (comb.)], [6.5 l/100 km (city)]...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
translator = Translator()

def detect_language(text):
    try:
        lang = translator.detect(text).lang
        return lang # tr
        # return LANGUAGES.get(lang) turkish
    except:
        return np.nan
    
def translate(x):
    try:
        text = x
        translated_text = translator.translate(text)
        return translated_text.text
    except: 
        return np.nan


def translateWithBlob(x,from_lang):
    try:
        blob=TextBlob(x)
        return blob.translate(from_lang=from_lang, to='en').string
    except: 
        return np.nan


In [55]:
## DONE
def removeWhiteFromList(x):
    return list(filter(lambda x: x.strip(), x))
df['desc'] = df['desc'].apply(lambda x: removeWhiteFromList(x) if isinstance(x, list) else x)

In [56]:
df.desc.apply(type).unique()

array([<class 'list'>, <class 'float'>], dtype=object)

In [59]:
# DONE convert to string
df['desc'] = df['desc'].apply(lambda x: '. '.join(x) if isinstance(x, list) else x)


In [86]:
df.desc.head(1)

0    Precio al contado: 16950 euros. Vehículo con H...
Name: desc, dtype: object

In [87]:
# DONE detect language
# df.desc.head(10).parallel_apply(lambda x: detect_language(x))
df['desc_lang'] = df.desc.parallel_apply(lambda x: detect_language(x))

In [62]:
df.desc.head(1)[0]

'Precio al contado: 16950 euros. Vehículo con HISTORIAL de mantenimiento y un solo propietario.. GARANTÍA de 12 meses ampliable a 24, consúltenos.. EXTRAS: Pantalla frontal, navegador, cámara de marcha atrás, espejos  eléctricos, volante multifunción, llantas, sensor lluvia y luces, cierre centralizado con mando, ordenador a bordo, aire acondicionado, ISOFIX, bluetooth, bluetooth Audio, USB y más extras.... Kilometraje GARANTIZADO y garantía de no haber sufrido accidente.. Precio final de venta, incluyendo IVA y no vinculado a financiación ni entrega de otro vehículo.. TRANSFERENCIA incluida en el precio .. IVA DEDUCIBLE.. Posibilidad de recogida de su antiguo vehículo como parte de pago.. Financiamos el 100% de nuestros vehículos. (consulte condiciones). Realizamos la gestión de búsqueda de tu vehículo sin ningún tipo de compromiso .. English Spoken!!!. <hr>LAS ESPECIFICACIONES DEL VEHÍCULO QUE SE INDICAN REFERENTES A : MODELO, EQUIPAMIENTO Y EXTRAS, PUEDEN CONTENER ERRORES, Y POR TAN

In [88]:
translate(df.desc.head(1)[0])

'Cash price: 16950 euros.Vehicle with maintenance history and a single owner .. 12 -month guarantee expandable to 24, consult us .. extras: front screen, browser, back camera, electric mirrors, multifunction steering wheel, tire, rain sensor and lights, centralized closing withCommand, computer on board, air conditioning, isofix, bluetooth, bluetooth audio, USB and more extras .... guaranteed mileage and guarantee of not having suffered accident .. final sale price, including VAT and not linked to financing or delivery ofanother vehicle .. transfer included in the price .. deductible VAT .. possibility of collection of your old vehicle as part of payment .. we finance 100% of our vehicles.(See conditions).We carry out the search management of your vehicle without any commitment .. English Sphen !!!<HR> vehicle specifications indicated by: model, equipment and extras, may contain errors, and therefore have no contractual binding character.Consult exposure .. financing conditions.See our

In [90]:
# DONE translate desc
# df['desc_en'] = df.desc.head(10).progress_apply(lambda x: translate(x)) # type: ignore
df['desc_en'] = df.desc.progress_apply(lambda x: translate(x)) # type: ignore

100%|██████████| 10/10 [00:01<00:00,  8.70it/s]


In [93]:
df.desc_en.head(10)[0]

'Cash price: 16950 euros.Vehicle with maintenance history and a single owner .. 12 -month guarantee expandable to 24, consult us .. extras: front screen, browser, back camera, electric mirrors, multifunction steering wheel, tire, rain sensor and lights, centralized closing withCommand, computer on board, air conditioning, isofix, bluetooth, bluetooth audio, USB and more extras .... guaranteed mileage and guarantee of not having suffered accident .. final sale price, including VAT and not linked to financing or delivery ofanother vehicle .. transfer included in the price .. deductible VAT .. possibility of collection of your old vehicle as part of payment .. we finance 100% of our vehicles.(See conditions).We carry out the search management of your vehicle without any commitment .. English Sphen !!!<HR> vehicle specifications indicated by: model, equipment and extras, may contain errors, and therefore have no contractual binding character.Consult exposure .. financing conditions.See our